<a href="https://colab.research.google.com/github/sjungmin98/study_data_analytics/blob/main/docs/quests/CDAs/CDA_fruitstore_saleshistory_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 한글 폰트 설치

In [2]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

## load dataset

In [3]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
df_LCOCT = pd.read_csv("fruitstore_saleshistory.csv")
df_LCOCT[:5]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09


## X(범주형) / Y(범주형) : 주로 독립성 증명
- 귀무가설 : 관계성이 없다
- 대립가설 : 관계성이 있다.

--> 연구원이나 분석가가 증명하고 싶은 것

#### 판매 채널 사용(X) (예: 매장 내 대비 온라인)
#### 요일(Y)


### 분석 내용 : 판매 채널 사용이 요일에 따라 차이가 있을까?
- 귀무가설 : 판매 채널 사용이 요일에 따라 차이가 없다. (0.05>p-value)
- 대립가설 : 판매 채널 사용이 요일에 따라 차이가 있다. (p-value<=0.05)

### 결측치 제거

In [19]:
df_LCOCT = df_LCOCT.dropna(subset=['구매매장', '요일'])

In [11]:
df_LCOCT_category_pivot = pd.pivot_table(df_LCOCT[['구매매장', '요일']]
               , index='구매매장', columns=['요일'], aggfunc=len)
df_LCOCT_category_pivot

요일,금,목,수,월,일,토,화
구매매장,,,,,,,
매장A,19296,18452,19243,21906,10060,17027,19427
매장B,24397,21829,22313,26604,12589,22306,23899
매장C,46037,43853,44188,52850,21851,40198,48001
매장D,20141,20010,19246,22442,9577,17960,19869


In [20]:
import scipy
import scipy.stats as stats
stats.chi2_contingency(df_LCOCT_category_pivot, correction=True)

Chi2ContingencyResult(statistic=393.8481798232706, pvalue=1.7528303477929499e-72, dof=18, expected_freq=array([[19528.90918278, 18510.96939075, 18661.34080057, 22005.06061332,
         9611.86138177, 17328.43867024, 19764.41996057],
       [23970.95703622, 22721.47654595, 22906.05145336, 27010.33414639,
        11798.17644007, 21269.96725631, 24260.03712171],
       [46245.19692278, 43834.67692408, 44190.76212033, 52108.81733518,
        22761.25195905, 41034.39937016, 46802.89536843],
       [20125.93685823, 19076.87713922, 19231.84562574, 22677.78790512,
         9905.7102191 , 17858.1947033 , 20368.64754929]]))

#### 분석 결과(기준 : p-value 0.05)
- 통계 측면 결론 : 대립 가설 성립, 귀무 가설 기각
- 사실 결론 : 판매 채널 사용이 요일에 따라 차이가 있다.